In [1]:
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.metrics.pairwise import cosine_similarity
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

# Calculation of Image Similarity with Feature Extraction Methods

In [2]:
# HOG özelliklerini çıkarma
def extract_hog_features(image):
    # Ensure the image is grayscale
    if len(image.shape) == 3:  # Check if the image is multi-channel
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    hog_features, _ = hog(image, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), block_norm='L2-Hys',
                          visualize=True)
    return hog_features

In [3]:
# SURF özelliklerini çıkarma
def extract_surf_features(image):
    surf = cv2.xfeatures2d.SURF_create()
    keypoints, descriptors = surf.detectAndCompute(image, None)
    return descriptors


In [4]:
def extract_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors

In [5]:
# LBP özelliklerini çıkarma
def extract_lbp_features(image):
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    n_bins = int(lbp.max() + 1)
    lbp_hist, _ = np.histogram(lbp, bins=n_bins, range=(0, n_bins), density=True)
    return lbp_hist

In [6]:
def pad_descriptors(descriptor1, descriptor2):
    max_length = max(descriptor1.shape[0], descriptor2.shape[0])
    padded_descriptor1 = np.zeros((max_length, descriptor1.shape[1]))
    padded_descriptor2 = np.zeros((max_length, descriptor2.shape[1]))
    padded_descriptor1[:descriptor1.shape[0], :] = descriptor1
    padded_descriptor2[:descriptor2.shape[0], :] = descriptor2
    return padded_descriptor1.flatten(), padded_descriptor2.flatten()

# Calculation of Image Similarity with SSIM and MSE

In [7]:
# SSIM benzerlik skoru hesaplama
def calculate_ssim(image1, image2):
    score, _ = ssim(image1, image2, full=True)
    return score

In [8]:
# MSE benzerlik skoru hesaplama
def calculate_mse(image1, image2):
    mse_score = mean_squared_error(image1.flatten(), image2.flatten())
    return 1 / (1 + mse_score)

In [9]:
# Histogram Similarity
def calculate_histogram_similarity(image1, image2):
    hist1 = cv2.calcHist([image1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([image2], [0], None, [256], [0, 256])
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return similarity

In [10]:
# ORB Feature Matching
def calculate_orb_similarity(image1, image2):
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    orb_similarity = len(matches)
    normalized_orb_similarity = orb_similarity / max(len(keypoints1), len(keypoints2))
    return normalized_orb_similarity

In [11]:
# İki resim arasındaki benzerlik skorunu hesaplama
def calculate_similarity(image1, image2):
    # HOG benzerlik skoru hesaplama
    hog_features1 = extract_hog_features(image1_gray)
    hog_features2 = extract_hog_features(image2_gray)
    hog_similarity = cosine_similarity([hog_features1], [hog_features2])[0][0]
    print(f'HOG similarity score: {hog_similarity}')

    # SIFT benzerlik skoru hesaplama
    sift_features1 = extract_sift_features(image1_gray)
    sift_features2 = extract_sift_features(image2_gray)
    sift_features1, sift_features2 = pad_descriptors(sift_features1, sift_features2)
    sift_similarity = cosine_similarity([sift_features1], [sift_features2])[0][0]
    print(f'SIFT similarity score: {sift_similarity}')

    # LBP benzerlik skoru hesaplama
    lbp_features1 = extract_lbp_features(image1_gray)
    lbp_features2 = extract_lbp_features(image2_gray)
    lbp_similarity = cosine_similarity([lbp_features1], [lbp_features2])[0][0]
    print(f'LBP similarity score: {lbp_similarity}')

    # Histogram benzerlik skoru hesaplama
    histogram_similarity = calculate_histogram_similarity(image1, image2)
    print(f'Histogram similarity score: {histogram_similarity}')

    # ORB benzerlik skoru hesaplama
    orb_similarity = calculate_orb_similarity(image1, image2)
    print(f'ORB similarity score: {orb_similarity}')

    # SSIM benzerlik skoru hesaplama
    ssim_score = calculate_ssim(image1, image2)
    print(f'SSIM similarity score: {ssim_score}')

    # MSE benzerlik skoru hesaplama
    mse_score = calculate_mse(image1, image2)
    print(f'MSE similarity score: {mse_score}')

    # Özellikleri birleştirme
    features1 = np.concatenate((hog_features1, sift_features1.flatten(), lbp_features1))
    features2 = np.concatenate((hog_features2, sift_features2.flatten(), lbp_features2))

    # Kosinüs benzerliğini hesaplama
    similarity = cosine_similarity([features1], [features2])
    return similarity[0][0]


In [14]:
# Resimleri yükleme
image1 = cv2.imread('images/test-images/img.png', cv2.IMREAD_COLOR)
image2 = cv2.imread('images/test-images/img.png', cv2.IMREAD_COLOR)

# Resimleri gri tonlamaya çevirme
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Benzerlik skorunu hesaplama
similarity_score = calculate_similarity(image1_gray, image2_gray)
print(f'The similarity score between the two images is: {similarity_score}')

HOG similarity score: 1.0
SIFT similarity score: 0.9999999999999983
LBP similarity score: 1.0000000000000002
Histogram similarity score: 1.0
ORB similarity score: 1.0
SSIM similarity score: 1.0
MSE similarity score: 1.0
The similarity score between the two images is: 1.0000000000000042


In [13]:
import cv2
print(cv2.__version__)

4.11.0
